# Lung Lobes Segmentation

## Imports

In [ ]:
%matplotlib inline

import SimpleITK as sitk
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp 
import gui
import cv2
import matplotlib.image as mpimg

# from mayavi import mlab
from scipy import signal
from myshow import myshow, myshow3d
from read_data import LoadData
from lung_segment import LungSegment
from vessel_segment import VesselSegment
from mpl_toolkits.mplot3d import Axes3D

## Read data

In [ ]:
data_path = "resource/"
img_name = "s1.mhd"
# img_name = "lola11-01.mhd"
data = LoadData(data_path, img_name)
data.loaddata()

In [ ]:
data_path = "resource/"
img_name = "s1map.mhd"
labelmap = LoadData(data_path, img_name)
labelmap.loaddata()

## Lung Lobes Segmentation
0. Data Preprocesing

In [ ]:
vs = VesselSegment(original=data.image, closing=labelmap.image)

In [ ]:
print "   Shrik the region of lung..."
vs.erosion(lunglabel=[201, 202])

print "   Pricessing Generate lung mask..."
vs.generate_lung_mask(offset = 1024)

# Write image...
Lung_mask = sitk.GetImageFromArray(vs.img)
sitk.WriteImage(Lung_mask, "Lung_mask.mhd")

print "   Processing Downsampling..."
vs.downsampling()

print "   Processing Thresholding..."
vs.thresholding(thval=180)

print "   Processing Region Growing..."
vs.max_filter(filter_size=5)

# print "   Processing Filtering..."
# vs.filtering(min_size=500, max_size=1000)

In [ ]:
filtered = sitk.GetImageFromArray(vs.temp_img)
sitk.WriteImage(filtered, "filtered.mhd")

In [ ]:
filtered = sitk.ReadImage("filtered.mhd")
filtered = sitk.GetArrayFromImage(filtered)
filtered[filtered > 0] = 1
binary_filtered = sitk.GetImageFromArray(filtered)
sitk.WriteImage(binary_filtered, "binary_filtered.mhd")

### Postprocessing for fissure enhancement

In [ ]:
import SimpleITK as sitk
from read_data import LoadData
import numpy as np
import collections

# data = LoadData(path="fissure_enhancement_cxx/", name="voxel_val_region_growing_3rd.mhd")
data = LoadData(path="", name="filtered_rg.mhd")
data.loaddata()
image = sitk.GetArrayFromImage(data.image)

In [ ]:
nonzeros = image[image > 0]
d = collections.Counter( nonzeros )
val_key = []
keys = set([])
for key, val in d.items():
    # if val > 1000:
    if val > 5000:
        keys.add(key)

In [ ]:
len(keys)

In [ ]:
image[image == 0] = 1

# for p in np.nditer(image, op_flags=['readwrite']):
#     if p.tolist() in keys:
#         p[...] = 0

for key in keys:
    image[image == key] = 0

image[image > 0] = 1
image[image == 0] = 255
image[image == 1] = 0

In [ ]:
img = sitk.GetImageFromArray(image.astype(np.uint8))

In [ ]:
sitk.WriteImage(img, "filtered.mhd")

In [ ]:
size = 7
closing = sitk.BinaryMorphologicalClosingImageFilter()
closing.SetForegroundValue(255)
closing.SetKernelRadius(size)
img = closing.Execute(img)

In [ ]:
sitk.WriteImage(img, "fissure_enhancement_cxx/voxel_val_region_growing_closing.mhd")

In [ ]:
data = LoadData(path="", name="binary_filtered.mhd")
data.loaddata()
vessel = sitk.GetArrayFromImage(data.image)

In [ ]:
fissure = sitk.GetArrayFromImage(img)

In [ ]:
import copy
fissure_vessel = copy.deepcopy(fissure)
fissure_vessel[fissure_vessel != 0] = 1
fissure_vessel[vessel != 0] = 2

In [ ]:
fissure_vessel_itk = sitk.GetImageFromArray(fissure_vessel)

In [ ]:
sitk.WriteImage(fissure_vessel_itk, "fissure_enhancement_cxx/fissure_vessel.mhd")

### Label map

In [ ]:
lung_mask = LoadData(path="", name="Lung_mask.mhd")
lung_mask.loaddata()
fissure = LoadData(path="fissure_enhancement_cxx/", name="voxel_val_region_growing_closing.mhd")
fissure.loaddata()
vessel = LoadData(path="", name="binary_filtered.mhd")
vessel.loaddata()

In [ ]:
lung_mask = sitk.GetArrayFromImage(lung_mask.image)
fissure = sitk.GetArrayFromImage(fissure.image)
vessel = sitk.GetArrayFromImage(vessel.image)

In [ ]:
lung_mask[lung_mask != 0] = 3
lung_mask[vessel > 0] = 1
lung_mask[fissure > 0] = 2

In [ ]:
lung_mask = sitk.GetImageFromArray(lung_mask)
sitk.WriteImage(lung_mask, "label_map.mhd")

### Fissure Evaluation

In [ ]:
result_dismap = LoadData(path="fissure_enhancement_cxx/", name="distmap_voxel_val_rg.mhd")
result_dismap.loaddata()
result_dismap_nda = sitk.GetArrayFromImage(result_dismap.image)

In [ ]:
gt_dismap = LoadData(path="fissure_enhancement_cxx/", name="distance_map_gt_fissure.mhd")
gt_dismap.loaddata()
gt_dismap_nda = sitk.GetArrayFromImage(gt_dismap.image)

In [ ]:
import copy

In [ ]:
gt_vals = copy.deepcopy(gt_dismap_nda)
gt_vals[result_dismap_nda == 0] = 0

In [ ]:
result_vals = copy.deepcopy(result_dismap_nda)
result_vals[gt_dismap_nda == 0] = 0

In [ ]:
num_total = float(np.count_nonzero(gt_vals) + np.count_nonzero(result_vals))
mean = float(np.sum(gt_vals) + np.sum(result_vals)) / num_total

In [ ]:
mean * 0.73